In [18]:
import time
import random
import requests
import pandas as pd
import time
import random

In [54]:
# MAIN
def save_checkpoint(data, checkpoint_number,shop_id):
    df = pd.DataFrame(data)
    df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'userid', 'shopid', 'comment', 'rating_star', 'like_count',  'model_name']]
    excel_filename = f'checkpoint_shop_{shop_id}_num_{checkpoint_number}.xlsx'
    df.to_excel('excel_filename', index=False)
    print(f"Checkpoint saved: {excel_filename}")


def get_reviews(shop_id,item_id):
    url = "https://shopee.com.my/api/v2/item/get_ratings"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    
    while True:
        params = {
            "itemid": item_id,
            "shopid": shop_id,
            "offset": offset,
            "limit": limit,
            "filter": "0",
            "flag": "0",
            "sort": "0",
            "append": "0",
            "before_bundle": "",
            "language": "en",
        }
    
        response = session.get(url, params=params).json()
        if response["error"]:
            print(f"Error: {response['error']}")
            break
        elif response["data"]["ratings"]:
            # print(offset)
            # print (response["data"]["ratings"])
            for rating in response["data"]["ratings"][:201]:
                model_name = rating.get('product_items')[0].get('model_name')
                rating['model_name'] = model_name
                reviews.append(rating)
                offset += limit
    
                time.sleep(random.randint(2,4))  
            # # save checkpoint data
                if len(reviews)%20==0:
                    print(len(reviews))
            #         save_checkpoint(reviews,len(reviews)//100,shop_id)
        else:
            break
        
    df = pd.DataFrame(reviews)  
    print(f"Retrieved {len(reviews)} reviews.")
    df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'userid', 'shopid', 'comment', 'rating_star', 'like_count',  'model_name']]
    print(f'Crawl shop: {shop_id}, item{item_id} completed')
    
    df.to_excel(f'Shopee_shopid{shop_id}.xlsx', index=False)

shop_id = "88201679"
item_id = "20493037179"

get_reviews(shop_id,item_id)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620


In [ ]:
import requests
import pandas as pd

url = "https://shopee.com.my/api/v2/item/get_ratings"
shop_id = "308461157"
item_id = "23354704062"
limit = 1
offset = 0
reviews = []

session = requests.Session()
session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})



while True:
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }

    response = session.get(url, params=params).json()
    if response["error"]:
        print(f"Error: {response['error']}")
        break
    elif response["data"]["ratings"]:
        print(offset)
        print (response["data"]["ratings"])

        for rating in response["data"]["ratings"]:
            model_name = rating.get('product_items')[0].get('model_name')
            rating['model_name'] = model_name
            reviews.append(rating)
            offset += limit

            time.sleep(random.randint(2,5))  
    else:
        break

df = pd.DataFrame(reviews)

print(f"Retrieved {len(reviews)} reviews.")
# extract 'skin suitability' and 'absorption' from 'comment' column
df[['skin_suitability', 'absorption']] = df['comment'].str.extract('Skin Suitability:(.*?)\\nAbsorption:(.*?)\\n', expand=True)

df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid', 'comment', 'rating_star', 'skin_suitability', 'absorption', 'model_name']]

print('success')

df.to_csv('ShopeeScrap1.csv', index=False)


In [43]:
import requests
import pandas as pd
import time
import random

def get_reviews(shop_id,item_id):
    url = "https://shopee.com.my/api/v2/item/get_ratings"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }
    response = session.get(url, params=params).json()
    print(response)
    for rating in response["data"]["ratings"][:10]:
        model_name = rating.get('product_items')[0].get('model_name')
        rating['model_name'] = model_name
        reviews.append(rating)
        offset += limit

    df = pd.DataFrame(reviews)
    print(f"Retrieved {len(reviews)} reviews.")
    
    df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'userid', 'shopid', 'comment', 'rating_star','like_count',  'model_name']]
    df.to_excel('temp.xlsx', index=False)

    
shop_id = "288286284"
item_id = "21993222972"
get_reviews(shop_id,item_id)

{'error_msg': None, 'data': {'ratings': [{'orderid': 153680409319298, 'itemid': 21993222972, 'cmtid': 13284042161, 'ctime': 1700535964, 'rating': 1, 'userid': 330045429, 'shopid': 288286284, 'comment': 'Chất lượng sản phẩm:tốt\nĐúng với mô tả:tốt\nTính năng nổi bật:màu titan tự nhiên\n\nShopdunk vận chuyển nhanh chóng, rất tốt. Mua ở đây yên tâm hơn mua ở apple flagship store nhiều. Sẽ ủng hộ mua của shopdunk', 'rating_star': 5, 'status': 2, 'mtime': 1704926407, 'editable': 0, 'opt': 2, 'filter': 7, 'mentioned': [], 'is_hidden': False, 'can_follow_up': None, 'follow_up': None, 'submit_time': 1700535964, 'author_username': 'h*****t', 'author_portrait': '', 'author_shopid': 330025854, 'anonymous': True, 'images': ['vn-11134103-7r98o-loc17m6vu9dj4c', 'vn-11134103-7r98o-loc17m6vx2if65'], 'videos': [], 'product_items': [{'itemid': 21993222972, 'shopid': 288286284, 'name': 'Apple iPhone 15 Pro Max 256GB', 'image': 'vn-11134207-7r98o-llt6eul6d7tr35', 'is_snapshot': 1, 'snapshotid': 1551599355

In [48]:
# product/
def get_reviews(shop_id,item_id):
    url = "https://shopee.com.my/api/v2/item/get_item_detail"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }
    keyword_search = 'iphone 15 pro max'
    url = "https://shopee.sg/api/v2/search_items/?by=relevancy&keyword={}&limit=100&newest=0&order=desc&page_type=search".format(keyword_search)
    response = session.get(url, params=params).json()
    print(response)
    
# can change "relevancy" to "latest": to sort by latest products instead

# Shopee API request
    #     if response["error"]:
    #         print(f"Error: {response['error']}")
    #         break
    #     elif response["data"]["ratings"]:
    #         print(offset)
    #         print (response["data"]["ratings"])
    
    #         for rating in response["data"]["ratings"]:
    #             model_name = rating.get('product_items')[0].get('model_name')
    #             rating['model_name'] = model_name
    #             reviews.append(rating)
    #             offset += limit
    
    #             time.sleep(random.randint(2,5))  
    #     else:
    #         break
    
    # df = pd.DataFrame(reviews)
    
    # print(f"Retrieved {len(reviews)} reviews.")
    # # extract 'skin suitability' and 'absorption' from 'comment' column
    # df[['skin_suitability', 'absorption']] = df['comment'].str.extract('Skin Suitability:(.*?)\\nAbsorption:(.*?)\\n', expand=True)
    
    # # select columns to keep in the final CSV file
    # df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid', 'comment', 'rating_star', 'skin_suitability', 'absorption', 'model_name']]
    # # df = df.to_string(index=False)
    
    # print('success')
    
    # # save the DataFrame as a CSV file
    df.to_csv('ShopeeScrap1.csv', index=False)
shop_id = "288286284"
item_id = "21993222972"
get_reviews(shop_id,item_id)

{'error': 'error_not_found'}
